# 2018-06-06 - Psychophysics using psychopy

On repasse sur un paradigme plus classique de psychophysique.

On récupère les initiales :

In [1]:
exp_info = input('Enter your initials : ')
print (exp_info)

2018-06-06:1606_LP


On défini la génération de MotionClouds :

In [2]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = '/tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [3]:
N_B_thetas = 5

B_thetas = np.pi*np.logspace(-7, 0, N_B_thetas, base=2)
print('B_thetas', B_thetas*180/np.pi)


B_thetas [  1.40625      4.73004234  15.90990258  53.51432018 180.        ]


In [4]:
N_B_thetas = 7

B_thetas = np.pi*np.logspace(-6, -1, N_B_thetas, base=2)
print('B_thetas', B_thetas*180/np.pi)


B_thetas [ 2.8125      5.01130529  8.92913092 15.90990258 28.34822362 50.51079217
 90.        ]


In [5]:
from psychopy import visual, core, event
import MotionClouds as MC

test_length = 150  # number of trials
#trial_length = 2.6  # length of trial, in second
fixation_length = .5  # length of fixation, in second
stim1_length = 0.25  # length of first stim, in second
stim2_length = 0.25  # length of the second stim, in second

fullscr = False

fullscr = True

# Psychopy window
win = visual.Window(fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
mouse = event.Mouse(visible=False)

fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
bitmap1 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap1.autolog = False
bitmap2 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap2.autolog = False
#bitmap3 = visual.ImageStim(win, '%s.png' %
#                           str(2 * (np.pi/3)), mask='circle')
#bitmap3.autolog = False


#msg = visual.TextStim(win, text='<- LEFT or RIGHT ->', color='black')
msg = visual.TextStim(win, text='?', color='black')

#shift_dict = {'right': 2,
#              'left':  3}


# Answer list
ans_list = []
    
std_theta = np.pi/8
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()
    
    # Shift to left (3) or to right (2) ?
    #shift = random.randint(2, 3)
    theta = np.clip(std_theta *  np.random.randn(), -np.pi/4, np.pi/4)

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    bitmap1.setImage(MC1)

    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2
    bitmap2.setImage(MC2)

    #MC3 = generate_random_cloud(2 * (np.pi/3), B_theta=B_theta)  # if shift = 3
    # MC to psychopy objects
    
    # Times the trial
    while clock.getTime() < fixation_length + stim1_length + stim2_length:
        if 0.0 <= clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length:
            bitmap1.draw()
        if fixation_length + stim1_length <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            bitmap2.draw()
        win.flip()

    # display orientation choice
    msg.draw()
    win.flip()

    # answer using the keyboard
    while True:
        ans = event.waitKeys()

        if len(ans) > 0:
            if ans[0] in ['escape', 'q']:
                win.close()
                core.quit()
            elif ans[0] in ['left', 'right']:
                correct = (np.sign(theta) > 0) and (ans[0]=='right')
                print('At trial ', trial, 'Angle=', '%3.3f' % (theta*180/np.pi), 'answer is ', ans[0], '(correct=', correct, '); bandwidth=', '%.3f' % (B_theta*180/np.pi))
                break

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, ans[0], B_theta])

win.close()

At trial  0 Angle= -7.539 answer is  left (correct= False ); bandwidth= 90.000
At trial  1 Angle= 4.973 answer is  right (correct= True ); bandwidth= 50.511
At trial  2 Angle= 39.822 answer is  right (correct= True ); bandwidth= 50.511
At trial  3 Angle= -13.869 answer is  left (correct= False ); bandwidth= 90.000
At trial  4 Angle= -14.159 answer is  left (correct= False ); bandwidth= 5.011
At trial  5 Angle= -6.366 answer is  left (correct= False ); bandwidth= 8.929
At trial  6 Angle= 11.276 answer is  right (correct= True ); bandwidth= 50.511
At trial  7 Angle= 39.018 answer is  right (correct= True ); bandwidth= 2.812
At trial  8 Angle= 31.096 answer is  right (correct= True ); bandwidth= 5.011
At trial  9 Angle= 17.056 answer is  right (correct= True ); bandwidth= 50.511
At trial  10 Angle= -45.000 answer is  left (correct= False ); bandwidth= 50.511
At trial  11 Angle= -6.477 answer is  left (correct= False ); bandwidth= 28.348
At trial  12 Angle= -20.475 answer is  left (correct

At trial  103 Angle= 3.809 answer is  right (correct= True ); bandwidth= 8.929
At trial  104 Angle= 23.903 answer is  right (correct= True ); bandwidth= 8.929
At trial  105 Angle= -21.331 answer is  left (correct= False ); bandwidth= 50.511
At trial  106 Angle= -36.415 answer is  left (correct= False ); bandwidth= 28.348
At trial  107 Angle= 1.140 answer is  right (correct= True ); bandwidth= 8.929
At trial  108 Angle= -7.377 answer is  left (correct= False ); bandwidth= 2.812
At trial  109 Angle= 6.042 answer is  right (correct= True ); bandwidth= 28.348
At trial  110 Angle= 18.639 answer is  right (correct= True ); bandwidth= 50.511
At trial  111 Angle= -5.219 answer is  left (correct= False ); bandwidth= 15.910
At trial  112 Angle= -2.711 answer is  left (correct= False ); bandwidth= 8.929
At trial  113 Angle= -6.399 answer is  right (correct= False ); bandwidth= 28.348
At trial  114 Angle= -16.127 answer is  left (correct= False ); bandwidth= 15.910
At trial  115 Angle= 33.997 answ

In [6]:
import pickle
pickle.dump(ans_list, open('./psychophysics_data/Psy_discrim_final_%s.p' % exp_info, 'wb'))